# Continuous Treatments and Repeated Actions
Julian Hsu
17-Dec-2023

The purpose of this script is to understand the biases of understanding repeated actions, where the repeated actions have selection bias. 

In [45]:
import numpy as np
import os as os
import pandas as pd

import statsmodels.api as sm

## Generate serially correlated action data

In [82]:
def repeated_action_data(N=200, timehorizon=3, treatments=[1,1,1,1]):
    ## Setup covariates
    X = np.random.normal(0,0.5, N)
    df = pd.DataFrame(data=X,columns=['x'])
    
    
    ## First action
    action = np.random.choice(2,N)        
    df['w0']=action
    
    ## Initialize outcome
    df['y'] = np.random.uniform(-1,1)*df['x'] +\
        np.random.normal(0,1) + treatments[0]*df['w0']

    
    ## Now allow for multiple downstream actions for a set number of times
    for t,te in zip( range(timehorizon), treatments[1:]):
        latent_action = 0.15*df['w{0}'.format(t)] + np.random.uniform(-1,1)*df['x']+np.random.normal(0,1,N)
        action = (latent_action > np.percentile(latent_action, 50) ) .astype(float)
        df['w{0}'.format(t+1)] = action
        df['y']+= df['w{0}'.format(t+1)] *te
    return df

In [83]:
df=repeated_action_data(N=500, timehorizon=2, treatments=[0.31,0.21,0.11])

In [84]:
sm.OLS(exog=sm.add_constant(df[['w0','w1','w2','x']]), 
      endog=df['y']).fit().summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.470e+31
Date:                Tue, 19 Dec 2023   Prob (F-statistic):               0.00
Time:                        07:34:34   Log-Likelihood:                 16610.
No. Observations:                 500   AIC:                        -3.321e+04
Df Residuals:                     495   BIC:                        -3.319e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9870    8.1e-17  -1.22e+16      0.000      -0.987      -0.987
w0             0.3100   8.16e-17    3.8e+15      0.000       0.310       0.310
w1             0.2100   8.25e-17   2.54e+15      0.000       0.210       0.210
w2             0.1100   8.79e-17   1.25e+15      0.000       0.110       0.110
x             -0.8871   9.13e-17  -9.72e+15      0.000      -0.887      -0.887
==============================================================================
Omnibus:                        4.083   Durbin-Watson:                   0.844
Prob(Omnibus):                  0.130   Jarque-Bera (JB):                4.046
Skew:                           0.220   Prob(JB):                        0.132
Kurtosis:                       2.991   Cond. No.                         3.86
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""